In [653]:
from keras import backend as K
from keras.engine.topology import Layer
import theano
import numpy as np

# _K = K.variable(np.array(
#             [
#             [[0,1,0,0],
#              [1,0,0,0],
#              [0,0,0,0],
#              [0,0,0,0]],
            
#             [[0,0,1,0],
#              [0,0,0,0],
#              [1,0,0,0],
#              [0,0,0,0]],
                
#             [[0,0,0,1],
#              [0,0,0,0],
#              [0,0,0,0],
#              [1,0,0,0]]
#             ]
#              ))
np_K = np.zeros((3,4,4))
#_K = K.zeros((3,4,4))
for i in range(0,3):
    print(i)
    np_K[i,0,i+1] = 1
    np_K[i,i+1,0] = 1
    
_K = K.variable(np_K)


def _lorentz(x, w):
    
    batch_size = K.shape(x)[0]
    vector_cluster_size = K.shape(w)[0]
    # w = K.zeros(3)
    # x = K.zeros(4)
    _mag = K.sqrt(K.sum(K.square(w), axis=1,keepdims=True))
    #assert _mag.ndim == 1
    #print("_w")
    #print(K.eval(w))
    #print("_mag")
    #print(K.eval(_mag))
    
    _g = 1.#/K.sqrt(1.-K.square(_mag))
    #assert _g.ndim == 1
    #print("_g")
    #print(K.shape(K.eval(_g)))
    #print(K.eval(_g))
    
    _inv_mag = 1/_mag
    #print("_inv_mag")
    #print(K.eval(_inv_mag))
    #_n = K.batch_dot(w,  _inv_mag)
    _n = w *  _inv_mag
    
    #print("_n")
    #print(K.eval(K.shape(_n)))
    #print(K.eval(_n))
    
    #print("_K")
    #print(K.eval(_K))
    _bK = K.reshape(_K, (1,3,4,4))
    
    #print("_bK")
    #print( K.eval(_bK))
    _bK = K.repeat_elements(_bK, vector_cluster_size, axis=0)
    
    #print("_bK")
    #print(K.eval(K.shape(_bK)))
    #print( K.eval(_bK))
    
    _bI = K.repeat_elements(K.reshape(K.eye(4), (1,4,4)), vector_cluster_size, axis=0)
    #print("_bI")
    #print(K.eval(K.shape(_bI)))
    
    _b1 = K.repeat_elements(K.eye(1),vector_cluster_size, axis=0)
    #print("_b1")
    #print(K.eval(K.shape(_b1)))
    #print("_g-_b1")
    #print(K.eval(K.shape(_g-_b1)))
    
    #assert _n.ndim == 2
    _nK = K.batch_dot(_n, _bK, axes=[[1],[1]])
    #_nK = _n * _K
    _nK = K.reshape(_nK, (vector_cluster_size,1,4,4))
    #print("_nK")
    #print(K.eval(K.shape(_nK)))
    #print(K.eval(_nK))
    
    #print("_g*_mag")
    #print(K.eval(K.shape(K.sum(_g*_mag, axis=1))))
    #r = K.batch_dot(_g*_mag, _nK, axes=[[1],[1]])
    #print(K.eval(r))
    #print(K.shape(K.eval(r)))
    #print(K.eval(K.batch_dot(K.sum(_g*_mag, axis=1), _nK)))
    
    #print("_nk2")
    #print(K.eval(K.batch_dot(_nK,_nK)))
    #K.repeat(K.eye(4), K.shape(w)[0])  -
    #*K.batch_dot(_nK,_nK)
    _nKr = K.reshape(_nK, (vector_cluster_size,1,4,4))
    _nK2r = K.reshape(K.batch_dot(_nK,_nK), (vector_cluster_size,1,4,4))
    _B = _bI - K.batch_dot(_g*_mag, _nKr, axes=[[1],[1]]) +K.batch_dot(_g-_b1,_nK2r,axes=[[1],[1]])
    
    #print("_B")
    #print(K.shape(K.eval(_B)))
    #print(K.eval(_B))
    
    x_shape = K.shape(x)
    x = K.reshape(x, (x_shape[0],x_shape[1], 1, 4))
    _B = K.reshape(_B, (1,vector_cluster_size,4,4))
    _mB = K.repeat_elements(_B,batch_size, axis=0)
    
    #print("_mB")
    #print(K.shape(K.eval(_mB)))
    #print(K.eval(_mB))
    #r = K.variable(np.random.random((50, 12,4)))
    #r = K.reshape(r, (50, 12,1,4))
    #print("r")
    #print(K.shape(K.eval(r)))
    #print(K.eval(r))
    
    #out = x + _mB
    #out = K.dot(_mB, x, axes=[[1],[1]])
    out = K.reshape(K.batch_dot(_mB, x, axes=[[1,3],[1,3]]), (batch_size, 1,4))
    
    #print("out")
    #print(K.shape(K.eval(out)))
    #print(K.eval(out))
    #f = lambda x_i: K.batch_dot(x_i, _B, axes=[[1],[1]])
    #for
    return out




class LorentzLayer(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LorentzLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        print(input_shape)
        ## the input shape should be (..., 4)
        # weights are including
        input_dim = input_shape[1]
        ## angle1, angle2, beta : dim 3
        initial_weight_value = np.random.random((input_dim,3))
        print(initial_weight_value.shape)
        initial_bias_value = np.random.random((1,3))
        print(initial_bias_value.shape)
        self.W = K.variable(initial_weight_value)
        self.B = K.variable(initial_bias_value)
        # self.trainable_weights = [self.W]
        self.trainable_weights = [self.W, self.B]

    def call(self, T, mask=None):
        print("call")
        print(type(T[0]))
        print(T[0])
        # should get a tensor (N,4) corresponding to n 4-vesctors
        #lorentzboost of the vectorial sum of each lorentzboosted 4 vector
        return _lorentz(_lorentz( T, self.W ), self.B)
        #return K.sum(T, axis=1)

    def get_output_shape_for(self, input_shape):
        print("get ouput shape")
        return (input_shape[0], self.output_dim)

0
1
2


In [654]:
# %load compile.py
#Compile
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
model = Sequential()
model.add(LorentzLayer(4 ,input_shape=(12,4)))
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

(None, 12, 4)
(12, 3)
(1, 3)
call
<class 'theano.tensor.var.TensorVariable'>
Subtensor{int64}.0
get ouput shape
call
<class 'theano.tensor.var.TensorVariable'>
Subtensor{int64}.0
get ouput shape


In [655]:
#X_train = [[(8, 1, 1, 1),
#          (7, 2, 1, 2),
#          (10, 1, 2, 1),
#          (16, 1, 1, 2),
#          (20, 3, 1, 1)]]
#y_train = [(15, 1, 1, 2)]

x_train = np.random.random((1000, 12, 4))
y_train = np.random.random((1000, 4))
print(x_train[0])
print(y_train[0])


[[  3.22298708e-01   8.21822361e-01   4.20888555e-01   1.84233083e-01]
 [  3.75308108e-01   7.12993626e-01   5.69778280e-04   5.50152228e-01]
 [  5.56942189e-01   8.86663236e-01   1.04850646e-01   9.20654928e-01]
 [  6.95596975e-01   2.57226106e-01   2.92598354e-01   1.61507268e-01]
 [  8.31834055e-01   1.69289018e-01   5.95563288e-01   7.58896548e-01]
 [  4.41004312e-01   8.32264725e-01   6.23875460e-01   8.35941442e-01]
 [  3.16786306e-01   5.31228389e-01   2.18218377e-01   2.79105613e-01]
 [  3.81873009e-01   1.95871543e-01   7.82164266e-01   3.03226405e-02]
 [  5.71220043e-01   7.75600515e-02   6.33320821e-01   9.89266780e-01]
 [  5.70398543e-02   4.80021424e-01   1.45255659e-01   6.34337581e-01]
 [  8.48047627e-01   5.96483578e-01   8.46676232e-01   7.71046195e-01]
 [  8.82264430e-01   5.57077439e-01   9.66980277e-01   6.28359901e-01]]
[ 0.37183971  0.63824208  0.16335966  0.44440239]


In [656]:
history = model.fit(x_train, y_train,
                    batch_size=50,
                    nb_epoch=100,
                       verbose=2)

INFO (theano.gof.compilelock): Refreshing lock /home/dweitekamp/.theano/compiledir_Linux-3.19--generic-x86_64-with-Ubuntu-15.04-vivid-x86_64-2.7.9-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /home/dweitekamp/.theano/compiledir_Linux-3.19--generic-x86_64-with-Ubuntu-15.04-vivid-x86_64-2.7.9-64/lock_dir/lock


Epoch 1/100
1s - loss: 6.3045 - acc: 0.2547
Epoch 2/100
0s - loss: 0.8196 - acc: 0.2519
Epoch 3/100
1s - loss: 0.7074 - acc: 0.2507
Epoch 4/100
1s - loss: 0.7006 - acc: 0.2496
Epoch 5/100
0s - loss: 0.6922 - acc: 0.2515
Epoch 6/100
0s - loss: 0.6855 - acc: 0.2500
Epoch 7/100
1s - loss: 0.6813 - acc: 0.2517
Epoch 8/100
0s - loss: 0.6753 - acc: 0.2527
Epoch 9/100
0s - loss: 0.6702 - acc: 0.2477
Epoch 10/100
0s - loss: 0.6670 - acc: 0.2488
Epoch 11/100
0s - loss: 0.6603 - acc: 0.2527
Epoch 12/100
0s - loss: 0.6562 - acc: 0.2516
Epoch 13/100
1s - loss: 0.6535 - acc: 0.2463
Epoch 14/100
0s - loss: 0.6476 - acc: 0.2506
Epoch 15/100
0s - loss: 0.6455 - acc: 0.2507
Epoch 16/100
1s - loss: 0.6411 - acc: 0.2502
Epoch 17/100
0s - loss: 0.6371 - acc: 0.2530
Epoch 18/100
0s - loss: 0.6381 - acc: 0.2498
Epoch 19/100
1s - loss: 0.6316 - acc: 0.2479
Epoch 20/100
0s - loss: 0.6283 - acc: 0.2524
Epoch 21/100
0s - loss: 0.6266 - acc: 0.2505
Epoch 22/100
1s - loss: 0.6216 - acc: 0.2528
Epoch 23/100
0s - l

In [657]:
2

2